In [1]:
import pandas as pd
import csv
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn import preprocessing
from sklearn import metrics
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout

Using TensorFlow backend.
C:\Users\zippy\anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\zippy\anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\zippy\anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\zippy\anaconda3\envs\te

In [2]:
# This function should transform data into a usable format 
def featureProcess(path):
    data = pd.read_csv(path)
    #get audio data from vect1 to vect148
    textData = data.loc[:, "tags"]
    
    return textData.values.tolist()

In [3]:
# This function should transform data into a usable format 
def labelProcess(path):
    data = pd.read_csv(path)
    #get classes
    labels = data.loc[:,"genre"].values.tolist()
    
    return labels

In [4]:
def toOneHot(classes, labels):
    one_hot_labels = []

    for label in labels:
        one_hot_vector = [0] * len(classes)
        one_hot_vector[classes.index(label)] = 1

        one_hot_labels.append(one_hot_vector)
    return one_hot_labels

In [5]:
def train(trainData, trainLabel, model):
    
    batch_size = 32  #the number of data to feed into model per batch
    num_epoch = 50  # go through your training data epoch times
    
    #callbacks stop training if val_loss is not improving.
    callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, verbose = 1)]
    
    history = model.fit(trainData, trainLabel, validation_split=0.2, epochs = num_epoch, batch_size=batch_size, verbose=1, callbacks = callbacks)
                 
    return history, model

In [6]:
def MLP(trainData, trainLabel):
    
    num_classes = len(trainLabel[0])  # total classes to train
    
    model = keras.models.Sequential()
    model.add(Dense(trainData.shape[1], input_shape = trainData.shape[1:], activation="relu", kernel_initializer="random_uniform"))
    model.add(Dropout(0.5))
    model.add(Dense(1000, activation="relu", kernel_initializer="random_uniform"))
    model.add(Dropout(0.5))
    model.add(Dense(500, activation="relu", kernel_initializer="random_uniform"))
    model.add(Dropout(0.5))
    model.add(Dense(100, activation="relu", kernel_initializer="random_uniform"))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation="softmax", kernel_initializer="random_uniform"))

    #Define loss, optimizer, and metrics
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [7]:
def predict(testData, model):
    prediction = model.predict(testData)
    return prediction

In [8]:
def evaluate(test_label, prediction):
    accuracy = metrics.accuracy_score(test_label, prediction)
    return accuracy

DATA Preprocess

In [9]:
trainData = np.array(featureProcess(r"D:\Unimelb\Introduction to Machine Learning\assignment2\dataset\dataset\train_features.csv"))
validData = np.array(featureProcess(r"D:\Unimelb\Introduction to Machine Learning\assignment2\dataset\dataset\valid_features.csv"))
testData = np.array(featureProcess(r"D:\Unimelb\Introduction to Machine Learning\assignment2\dataset\dataset\test_features.csv"))

In [11]:
#create BOW
trainTxt = []
validTxt = []
testTxt = []
for i in range(len(trainData)):
    trainTxt.append(trainData[i].replace(",", ""))
    
for i in range(len(validData)):
    validTxt.append(validData[i].replace(",", ""))   
    
for i in range(len(testData)):
    testTxt.append(testData[i].replace(",", ""))   
    
vectorizer = CountVectorizer() 
trainVectors = vectorizer.fit_transform(trainTxt)
validVectors = vectorizer.transform(validTxt)
testVectors = vectorizer.transform(testTxt)

In [12]:
trainLabel = labelProcess(r"D:\Unimelb\Introduction to Machine Learning\assignment2\dataset\dataset\train_labels.csv")
validLabel = labelProcess(r"D:\Unimelb\Introduction to Machine Learning\assignment2\dataset\dataset\valid_labels.csv")
#get classes
classes = list(set(trainLabel))

oneHotTrain = np.array(toOneHot(classes, trainLabel))
oneHotvalid = np.array(toOneHot(classes, validLabel))

Training

In [13]:
model = MLP(trainVectors, oneHotTrain)
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 4827)              23304756  
_________________________________________________________________
dropout_1 (Dropout)          (None, 4827)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 1000)              4828000   
_________________________________________________________________
dropout_2 (Dropout)          (None, 1000)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 500)               500500    
_________________________________________________________________
dropout_3 (Dropout)  

In [14]:
history = train(trainVectors, oneHotTrain, model)
print("Finish training")

Instructions for updating:
Use tf.cast instead.
Train on 6142 samples, validate on 1536 samples
Epoch 1/50
6142/6142 [==============================] - 7s 1ms/step - loss: 1.7624 - acc: 0.3211 - val_loss: 1.5173 - val_acc: 0.4388
Epoch 2/50
6142/6142 [==============================] - 4s 615us/step - loss: 1.3866 - acc: 0.5016 - val_loss: 1.4633 - val_acc: 0.4876
Epoch 3/50
6142/6142 [==============================] - 4s 618us/step - loss: 1.1317 - acc: 0.5899 - val_loss: 1.6273 - val_acc: 0.4870
Epoch 4/50
6142/6142 [==============================] - 4s 614us/step - loss: 0.8918 - acc: 0.6744 - val_loss: 1.8730 - val_acc: 0.4759
Epoch 5/50
6142/6142 [==============================] - 4s 616us/step - loss: 0.6816 - acc: 0.7595 - val_loss: 2.0965 - val_acc: 0.4870
Epoch 00005: early stopping
Finish training


In [15]:
loss, accuracy = model.evaluate(validVectors, oneHotvalid)
print("loss, accuracy:", loss, accuracy)

450/450 [==============================] - 0s 113us/step
loss, accuracy: 1.5516943714353773 0.5688888888888889


Predict

In [ ]:
trackID = pd.read_csv(r"D:\Unimelb\Introduction to Machine Learning\assignment2\dataset\dataset\test_features.csv").loc[:, "trackID"].values.tolist()
prediction = predict(testVectors, model)

In [ ]:
prediction = prediction.tolist()
results_name = []

for result in prediction:
    results_name.append(classes[result.index(max(result))])
    
result = list(zip(trackID,results_name))

In [ ]:
with open("BOW-MLP_outPut.csv", 'w', newline='') as csvFile:
    writer = csv.writer(csvFile)
    writer.writerow(["trackID", "genre"])
    writer.writerows(result)